In [1]:
import os
import numpy as np
from mpc_utils import load_train_test_data_27s, load_train_test_data
import torch
from models.backpropamine_torch import BackpropamineRNN
from tqdm import tqdm
import time

In [2]:
class Logger:
    def __init__(self, log_folder, update_freq=10) -> None:
        self.log_folder = log_folder
        os.makedirs(log_folder, exist_ok=True)
        self.datastore = {}
        self.counter = {}
        self.update_freq = update_freq

    def write_scalar(self, scalar: float, filename: str, update_freq=None):
        self.datastore[filename] = self.datastore.get(filename, []) + [scalar]
        # update every self.update_freq steps
        self.counter[filename] = self.counter.get(filename, 0) + 1
        ufreq = update_freq if update_freq is not None else self.update_freq
        if self.counter[filename] >= ufreq:
            self.save()
            self.counter[filename] = 0

    def save(self):
        for filename, data in self.datastore.items():
            np.savetxt(f'./{self.log_folder}/{filename}.txt', np.array(data))
            # np.save(f'./{self.log_folder}/{filename}.npy', np.array(data))

    def close(self):
        self.save()

In [3]:
x_train, y_train, x_test, y_test = load_train_test_data_27s(include_L=True)
l_train, l_test = x_train[:, :, 7:], x_test[:, :, 7:]
x_train, x_test = x_train[:, :, :7], x_test[:, :, :7]

x = np.concatenate([x_train, x_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)
l = np.concatenate([l_train, l_test], axis=0)
low_thr = 0.12
up_thr = 0.415
tr_idx = (l[:,0,0] > low_thr) & (l[:,0,0] < up_thr)
te_idx = ~tr_idx
x_train, y_train, l_train = x[tr_idx], y[tr_idx], l[tr_idx]
x_test, y_test, l_test = x[te_idx], y[te_idx], l[te_idx]

x_train.shape, y_train.shape, x_test.shape, y_test.shape, l_train.shape

((192, 1350, 7), (192, 1350, 1), (33, 1350, 7), (33, 1350, 1), (192, 1350, 1))

In [4]:
from models.lmu_torch import LMUCell

DEVICE = 'cpu'

class BackpropamineLMU(torch.nn.Module):
    def __init__(self, input_size, lmu_hidden_size, lmu_memory_size, 
                 lmu_theta, rnn_hidden_size, output_size, 
                 plasticity_noise=None, device='cpu'):
        super(BackpropamineLMU, self).__init__()
        self.lmu_cell = LMUCell(
            input_size=input_size, hidden_size=lmu_hidden_size, 
            memory_size=lmu_memory_size, theta=lmu_theta
        )
        self.bp_rnn = BackpropamineRNN(
            isize=lmu_hidden_size, hsize=rnn_hidden_size, 
            osize=output_size, random_plasticity=plasticity_noise is not None,
            plasticity_noise=plasticity_noise, device=DEVICE
        )
        self.device = device
        self.lmu_cell.to(device)
        self.bp_rnn.to(device)
    
    def forward(self, x, hidden):
        # assume x is [batch_size, seq_len, input_size]
        # hidden = (lmu_state, bp_hid)
        T = x.shape[1]
        if hidden is None:
            lmu_hid = torch.zeros(x.shape[0], self.lmu_cell.hidden_size, device=self.device)
            lmu_mem = torch.zeros(x.shape[0], self.lmu_cell.memory_size, device=self.device)
            lmu_state = (lmu_hid, lmu_mem)
            bp_hid = self.bp_rnn.initialZeroStateHebb(x.shape[0])
        else:
            lmu_state, bp_hid = hidden
        outputs = []
        for t in range(T):
            # print('x', x.shape)
            # print('lmu_state', lmu_state.shape)
            lmu_state = self.lmu_cell(x[:,t,:], lmu_state)
            out_t, bp_hid = self.bp_rnn(lmu_state[0], bp_hid) # pass lmu_state[0] = lmu_hid
            outputs.append(out_t)
        return torch.stack(outputs, dim=1), (lmu_state, bp_hid)

In [5]:
def train(epochs, model, logger, x_train, y_train, n_batches, batch_size, optimizer, loss, device):
    for epoch in range(epochs):
        epoch_start = time.time()
        hidden = None

        epoch_loss = []
        model.train()
        for batch_idx in range(n_batches-1):
            if hidden is not None:
                # reset hebbian hidden state
                hidden = (hidden[0], model.bp_rnn.initialZeroStateHebb(batch_size))

            x = x_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
            y = y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
            x = torch.tensor(x, dtype=torch.float32, device=device)
            y = torch.tensor(y, dtype=torch.float32, device=device)

            optimizer.zero_grad()

            y_pred, hidden = model(x, None)

            l = loss(y_pred, y)
            l.backward()
            optimizer.step()
            optimizer.zero_grad()

            epoch_loss.append(l.item())

        avg_epoch_loss = np.array(epoch_loss).mean()
        logger.write_scalar(avg_epoch_loss, 'train_loss')
        logger.write_scalar(time.time()-epoch_start, 'ep_dur')

        if epoch % 10 == 9 or epoch in [0,1,2]:
            print(epoch+1, avg_epoch_loss, time.time()-epoch_start)
        else:
            print(epoch+1, avg_epoch_loss, time.time()-epoch_start, end='\r')

In [6]:
log_folder = f'logs/mpc_plasticity/rp05x10-01x20_{time.strftime("%m%d_%H%M", time.gmtime())}'
logger = Logger(log_folder, update_freq=1)

In [7]:
loss = torch.nn.MSELoss()
batch_size = 32
n_batches = x_train.shape[0] // batch_size

In [8]:
model = BackpropamineLMU(input_size=7, lmu_hidden_size=128, lmu_memory_size=128,
                         lmu_theta=100, rnn_hidden_size=128, output_size=1, plasticity_noise=0.05,
                         device=DEVICE)
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())
epochs = 10
train(epochs, model, logger, x_train, y_train, n_batches, batch_size, optimizer, loss, DEVICE)

1 0.4832513093948364 32.22351408004761
2 0.32660278081893923 32.0911009311676
3 0.33275737166404723 31.59447979927063
10 0.30536890029907227 31.65047287940979


In [9]:
model.bp_rnn.plasticity_noise = 0.01
optimizer = torch.optim.Adam(model.parameters())
epochs = 20
train(epochs, model, logger, x_train, y_train, n_batches, batch_size, optimizer, loss, DEVICE)

1 0.29240294694900515 31.141382932662964
2 0.2666308730840683 31.36859703063965
3 0.2577412873506546 31.405122756958008
10 0.053297373652458194 109.45631194114685
20 0.02183917872607708 28.869108438491825


In [10]:
torch.save(model.state_dict(), f'{log_folder}/model_pre.pt')

**TODO: freeze all weights except plasticity**

In [18]:
model = BackpropamineLMU(input_size=7, lmu_hidden_size=128, lmu_memory_size=128,
                         lmu_theta=100, rnn_hidden_size=128, output_size=1, plasticity_noise=None,
                         device=DEVICE)
model.to(DEVICE)
# load model with random plasticity - insert new value for plasticity matrix
x = torch.load(f'{log_folder}/model.pt')
x['bp_rnn.alpha'] = torch.rand(128, 128) * 0.001
model.load_state_dict(x)
# train final model
optimizer = torch.optim.Adam(model.parameters())
epochs = 100
train(epochs, model, logger, x_train, y_train, n_batches, batch_size, optimizer, loss, DEVICE)

NameError: name 'train' is not defined

## logs

- random_plasiticy 0.01 works fine (losses per epoch: 0.22, 0.10, 0.06, 0.03)
- random_plasticity 0.05 works as well (losses per epoch: 0.37, 0.33, 0.32, 0.30)
- random_plasticity 0.05 for 10 epochs, then 0.01 for 10 epochs -> final loss 